<a href="https://colab.research.google.com/github/RomuloRangel/projeto2023_02/blob/main/Projeto2023_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas necessarias

In [1]:
import pandas as pd
import numpy as np

In [ ]:
#Definirei uma importação via URL
url = ('')
df = pd.read_csv(url)